In [ ]:
import pandas as pd
import json
from pymongo import MongoClient

In [ ]:
df = pd.read_csv("pay_gap_Europe.csv")
df

In [ ]:
def transform_csv_to_json(cvs_file:str):
    pass
    # Read the csv

    # clean and transform the data => for example, remove white spaces from column names and convert them to lowercase


    
    # Convert to expected data types
    

    #Create a list of sectors columns
    
    # Convert sector columns to float
   
   # Fill missing values with "Null" 

    # Transform the data to the expected format according to the schema

    # Return the result
    



In [ ]:

csv_file = "pay_gap_Europe.csv"  

transformed_data = transform_csv_to_json(csv_file)

# Print first document as a sample
print(json.dumps(transformed_data[:1], indent=4))

In [ ]:
    # Save as JSON file
with open("output.json", "w", encoding="utf-8") as f:
        json.dump(transformed_data, f, indent=4, ensure_ascii=False)

In [ ]:
def upload_to_mongodb(json_file, db_name="paydata", collection_name="countries"):
    """
    Uploads the transformed JSON data into MongoDB.

    Parameters:
        json_file (str): Path to the JSON file containing the transformed data.
        db_name (str): MongoDB database name.
        collection_name (str): MongoDB collection name.
    
    Returns:
        int: Number of inserted documents.
    """

    # Connect to MongoDB (Modify connection string if needed)
    db_string = ""

    client = MongoClient(db_string)  
    db = client[db_name]
    collection = db[collection_name]

    # Load JSON file
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Insert or update documents (upsert to avoid duplicates)
    

    print(f"Inserted/Updated {len(data)} documents into MongoDB.")
    return len(data)



In [ ]:
upload_to_mongodb("output.json","PayGapDB","PayGapCollection")